# Experiment 2: SHAP values in different poisoning percentages

This experiment should visualize the effect of label flipping on SHAP values.

Experimental Setup: 
* number of clients: (1, 100, 200)
* dataset size respectively: (60000, 600, 300)
* percentage (0, 0.1, 0.2, 0.25, 0.3, 0.4, 0.425, 0.45, 0.475, 0.5, 0.6, 1)

In [1]:
from federated_learning.utils import SHAPUtil
from federated_learning import ClientPlane, Configuration

In [2]:
class ObserverConfiguration():
    experiment_type = "data_poisoning_shap"
    experiment_id = 0
    test = False
    dataset_type = "MNIST"
    # Client Configurations 
    client_name = "client"
    client_type = "client"

In [3]:
config = Configuration()
data = config.DATASET(config)
shap_util = SHAPUtil(data.test_dataloader)
observer_config = ObserverConfiguration()

MNIST training data loaded.
MNIST test data loaded.


In [ ]:
for i in range(10, 20, 1):
    observer_config.experiment_id = i
    for number_client in [1, 200]:
        config.NUMBER_OF_CLIENTS = number_client
        client_plane = ClientPlane(config, observer_config, data, shap_util)
        for percentage in [0, 0.1, 0.2, 0.25, 0.3, 0.4, 0.425, 0.45, 0.475, 0.5, 0.6, 1]:
            config.DATA_POISONING_PERCENTAGE = percentage
            client_plane.update_config(config, observer_config)
            client_plane.poison_clients()
            client_plane.clients[0].test()
            for epoch in range(1, config.N_EPOCHS + 1):
                client_plane.clients[0].train(epoch)
                client_plane.clients[0].test()
            client_plane.clients[0].push_metrics()
            client_plane.reset_client_nets()
            client_plane.reset_poisoning_attack()